In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import osmnx as ox
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

In [ ]:
DATA_PATH = "../../data/"
TAXI_DATA_PATH = "../../data/taxi_after_proc/clean202006"
DATASET = "sz_taxi_202006"

MIN_LAT = 22.5311
MAX_LAT = 22.5517
MIN_LNG = 114.0439
MAX_LNG = 114.0633

DATE_PREFIX = "2020-06-"
START_DAY = 1
END_DAY = 30

DOWNSAMPLING_INTERVAL = 30
TRAJ_SPLIT_INTERVAL = 600
FLOW_AGG_INTERVAL_MINUTE = 15

geo

---

In [2]:
df_edges = gpd.GeoDataFrame.from_file(os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "edges.shp"))

df_edges

,u,v,key,osmid,oneway,lanes,highway,length,name,maxspeed,access,ref,tunnel,bridge,fid,geometry
0,296024760,6033481369,0,640465230,1,1,trunk_link,165.507,None,None,None,None,None,None,0,"LINESTRING (114.05928 22.53152, 114.05977 22.5..."
1,409392019,6033481329,0,40971705,1,4,primary,47.579,滨河大道辅路,40,yes,None,None,None,1,"LINESTRING (114.06133 22.53186, 114.06087 22.5..."
2,499236929,499264828,0,40971702,1,None,primary_link,58.461,None,None,None,None,None,None,2,"LINESTRING (114.04445 22.53283, 114.04441 22.5..."
3,499237442,499256522,0,684241517,1,None,trunk_link,177.073,None,None,None,None,None,None,3,"LINESTRING (114.04560 22.53130, 114.04732 22.5..."
4,499237442,7232025523,0,328724256,1,None,trunk,478.781,滨河大道,None,None,S550,None,None,4,"LINESTRING (114.04560 22.53130, 114.04758 22.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,8516154911,8516154889,0,26274669,1,4,primary,156.629,彩田路,None,None,None,None,None,403,"LINESTRING (114.06284 22.53753, 114.06284 22.5..."
404,8516154912,8516154910,0,41019601,1,4,primary,16.481,彩田路,None,None,None,None,None,404,"LINESTRING (114.06300 22.53753, 114.06300 22.5..."
405,8869146293,7558081682,0,26274669,1,4,primary,33.430,彩田路,None,None,None,None,None,405,"LINESTRING (114.06291 22.53384, 114.06291 22.5..."
406,8869146293,7959990521,0,617224846,1,2,tertiary,93.933,福华五路,None,None,None,None,None,406,"LINESTRING (114.06291 22.53384, 114.06279 22.5..."


In [5]:
df_geo=pd.DataFrame()

df_geo["geo_id"]=df_edges["fid"]
df_geo["type"]="LineString"
df_geo["coordinates"]=df_edges["geometry"].apply(lambda x: list(x.coords))

df_geo

,geo_id,type,coordinates
0,0,LineString,"[(114.0592761, 22.5315228), (114.0597732, 22.5..."
1,1,LineString,"[(114.0613304, 22.5318565), (114.0608672, 22.5..."
2,2,LineString,"[(114.0444457, 22.5328302), (114.0444143, 22.5..."
3,3,LineString,"[(114.0456007, 22.5312996), (114.0473224, 22.5..."
4,4,LineString,"[(114.0456007, 22.5312996), (114.0475796, 22.5..."
...,...,...,...
403,403,LineString,"[(114.0628409, 22.537532), (114.062843, 22.537..."
404,404,LineString,"[(114.0630035, 22.5375338), (114.0630009, 22.5..."
405,405,LineString,"[(114.0629074, 22.5338432), (114.0629128, 22.5..."
406,406,LineString,"[(114.0629074, 22.5338432), (114.0627946, 22.5..."


In [6]:
df_geo.to_csv(os.path.join(DATA_PATH, DATASET, f"{DATASET}.geo"), index=False)

rel

---

In [58]:
line_graph=nx.read_gpickle(os.path.join(DATA_PATH, DATASET, "line_graph_drive.pkl"))

len(line_graph.nodes)

408

In [44]:
# unziped_node_list=list(zip(*list(line_graph.nodes)))

# unziped_node_list[2].index(1)
# list(line_graph.nodes)[53]
# df_edges.loc[(df_edges["u"]==7959990623) & (df_edges["v"]==7959990621)]
# df_edges.loc[df_edges["key"]==1]

相当于两个表 join on (u, v, key)

In [53]:
df_right=df_edges[["u", "v", "key", "fid"]]

unziped_node_list=list(zip(*list(line_graph.nodes)))
df_left=pd.DataFrame()
df_left["u"]=unziped_node_list[0]
df_left["v"]=unziped_node_list[1]
df_left["key"]=unziped_node_list[2]
df_left["id"]=range(len(unziped_node_list[0]))

df_left
df_right

,u,v,key,id
0,296024760,6033481369,0,0
1,6033481369,5435020958,0,1
2,409392019,6033481329,0,2
3,6033481329,7959990534,0,3
4,6033481329,7959990557,0,4
...,...,...,...,...
403,8869146293,7959990521,0,403
404,8516154912,8516154910,0,404
405,8516154910,8516154909,0,405
406,8516154910,8875446876,0,406


,u,v,key,fid
0,296024760,6033481369,0,0
1,409392019,6033481329,0,1
2,499236929,499264828,0,2
3,499237442,499256522,0,3
4,499237442,7232025523,0,4
...,...,...,...,...
403,8516154911,8516154889,0,403
404,8516154912,8516154910,0,404
405,8869146293,7558081682,0,405
406,8869146293,7959990521,0,406


In [54]:
df_join=pd.merge(df_left, df_right, on=["u", "v", "key"])

df_join

,u,v,key,id,fid
0,296024760,6033481369,0,0,0
1,6033481369,5435020958,0,1,86
2,409392019,6033481329,0,2,1
3,6033481329,7959990534,0,3,84
4,6033481329,7959990557,0,4,85
...,...,...,...,...,...
403,8869146293,7959990521,0,403,406
404,8516154912,8516154910,0,404,404
405,8516154910,8516154909,0,405,400
406,8516154910,8875446876,0,406,401


In [55]:
adj=nx.convert_matrix.to_numpy_array(line_graph)

rel=[]
rel_id_counter=0

for i in range(len(adj)):
    for j in range(len(adj[0])):
        if adj[i][j]:
            rel.append([rel_id_counter, "geo", df_join.iloc[i]["fid"], df_join.iloc[j]["fid"]])
            rel_id_counter+=1

df_rel=pd.DataFrame(rel, columns=["rel_id", "type", "origin_id", "destination_id"])

df_rel

,rel_id,type,origin_id,destination_id
0,0,geo,0,86
1,1,geo,86,77
2,2,geo,1,84
3,3,geo,1,85
4,4,geo,84,269
...,...,...,...,...
682,682,geo,404,401
683,683,geo,400,398
684,684,geo,400,399
685,685,geo,401,407


In [57]:
# df_rel.to_csv(os.path.join(DATA_PATH, DATASET, f"{DATASET}.rel"), index=False)

dyna

---

In [3]:
df_fmm_res=pd.read_csv(os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "mr.txt"), sep=";")
df_fmm_data=pd.read_csv(os.path.join(DATA_PATH, DATASET, f"fmm_{DATASET}", "gps.csv"), sep=";", parse_dates=["time"])

df_fmm_res
df_fmm_data

,id,opath,cpath,mgeom
0,89066,"4,103","4,103","LINESTRING(114.04962207 22.5313713813,114.0502..."
1,67198,"53,374","53,371,375,374","LINESTRING(114.044403636 22.5352034025,114.044..."
2,40638,"4,103,103","4,103","LINESTRING(114.0456007 22.5312996,114.0475796 ..."
3,7815,"248,216","248,246,216","LINESTRING(114.057810976 22.5396186225,114.057..."
4,90628,"19,349,105","19,47,345,350,349,105","LINESTRING(114.05163215 22.5324007076,114.0515..."
...,...,...,...,...
250553,243445,"87,87,364,364,78,25","87,37,38,96,115,117,124,296,364,175,185,186,19...","LINESTRING(114.056488653 22.5316163843,114.053..."
250554,243446,"209,206","209,207,205,206","LINESTRING(114.05259555 22.5389715897,114.0526..."
250555,243447,"216,46,157,244,79,324,255,255,306,259","216,46,190,245,188,187,154,157,72,241,244,74,2...","LINESTRING(114.054046844 22.5395489662,114.053..."
250556,243448,"26,5","26,52,134,147,178,162,107,106,361,299,297,368,...","LINESTRING(114.044500177 22.5389746223,114.044..."


,id,x,y,time
0,0,114.047810,22.539036,2019-12-02 00:51:17
1,0,114.047740,22.538887,2019-12-02 00:51:47
2,0,114.048020,22.538324,2019-12-02 00:53:32
3,0,114.044430,22.533690,2019-12-02 00:55:17
4,1,114.063286,22.537853,2019-12-02 13:55:29
...,...,...,...,...
1000333,250557,114.049310,22.539335,2019-12-06 09:46:14
1000334,250557,114.047485,22.537493,2019-12-06 09:47:28
1000335,250557,114.044470,22.534008,2019-12-06 09:48:43
1000336,250557,114.044530,22.531244,2019-12-06 09:49:13


In [4]:
df_fmm_res[df_fmm_res.isna().any(axis=1)]

df_fmm_res=df_fmm_res.dropna()
df_fmm_res.isna().sum()

,id,opath,cpath,mgeom
119,9380,NaN,NaN,LINESTRING()
192,60952,NaN,NaN,LINESTRING()
440,6257,NaN,NaN,LINESTRING()
652,18764,NaN,NaN,LINESTRING()
1099,17209,NaN,NaN,LINESTRING()
...,...,...,...,...
250123,213421,NaN,NaN,LINESTRING()
250162,245704,NaN,NaN,LINESTRING()
250266,245739,NaN,NaN,LINESTRING()
250311,243349,NaN,NaN,LINESTRING()


id       0
opath    0
cpath    0
mgeom    0
dtype: int64

In [5]:

N=len(line_graph.nodes)

dyna_file=open(os.path.join(DATA_PATH, DATASET, f"{DATASET}.dyna"), "w")
write_length=dyna_file.write("dyna_id,type,time,entity_id,flow\n") # disable printing return value

flow_matrix=np.zeros((END_DAY - START_DAY + 1, 24 * 60 // FLOW_AGG_INTERVAL_MINUTE, N), dtype=np.int16)

for traj_id in tqdm(df_fmm_res["id"].values):
    time_list=df_fmm_data.loc[df_fmm_data["id"]==traj_id]["time"].values
    road_list=np.array(df_fmm_res.loc[df_fmm_res["id"]==traj_id]["opath"].values[0].split(","), dtype=np.int16)
    
    assert(len(time_list)==len(road_list))
    
    for i in range(len(road_list)):
        time_i=pd.to_datetime(time_list[i])
        day=time_i.day
        mins=time_i.hour*60+time_i.minute
        
        flow_matrix[day-START_DAY][mins//FLOW_AGG_INTERVAL_MINUTE][road_list[i]]+=1
        
dyna_id_counter=0   
for day in range(flow_matrix.shape[0]):
    for interval in range(flow_matrix.shape[1]):
        for road in range(flow_matrix.shape[2]):
            write_length=dyna_file.write(f"{dyna_id_counter},"+
                    "state,"+
                    f"{DATE_PREFIX}{str(day+START_DAY).zfill(2)}T{str(interval*FLOW_AGG_INTERVAL_MINUTE//60).zfill(2)}:{str((interval%(60//FLOW_AGG_INTERVAL_MINUTE))*FLOW_AGG_INTERVAL_MINUTE).zfill(2)}:00Z,"+
                    f"{road},"+
                    f"{flow_matrix[day][interval][road]}\n")
            dyna_id_counter+=1
            
dyna_file.close()

100%|██████████| 248216/248216 [09:27<00:00, 437.11it/s]


In [4]:
df_dyna=pd.read_csv(os.path.join(DATA_PATH, DATASET, f"{DATASET}.dyna"))

df_dyna

,dyna_id,type,time,entity_id,flow
0,0,state,2019-12-02T00:00:00Z,0,11
1,1,state,2019-12-02T00:00:00Z,1,20
2,2,state,2019-12-02T00:00:00Z,2,5
3,3,state,2019-12-02T00:00:00Z,3,7
4,4,state,2019-12-02T00:00:00Z,4,25
...,...,...,...,...,...
195835,195835,state,2019-12-06T23:45:00Z,403,0
195836,195836,state,2019-12-06T23:45:00Z,404,0
195837,195837,state,2019-12-06T23:45:00Z,405,0
195838,195838,state,2019-12-06T23:45:00Z,406,0


config

---

In [6]:
import json

config={}

config["geo"]={}
config["geo"]["including_types"]=["LineString"]
config["geo"]["LineString"]={}

config["rel"]={}
config["rel"]["including_types"]=["geo"]
config["rel"]["geo"]={}

config["dyna"]={}
config["dyna"]["including_types"]=["state"]
config["dyna"]["state"]={"entity_id": "geo_id", "flow": "num"}

config["info"]={}
config["info"]["data_files"]=DATASET
config["info"]["geo_file"]=DATASET
config["info"]["rel_file"]=DATASET
config["info"]["data_col"]=["flow"]
config["info"]["output_dim"]=1
# config["info"]["weight_col"]="link_weight"
config["info"]["time_intervals"]=60*FLOW_AGG_INTERVAL_MINUTE
config["info"]["init_weight_inf_or_zero"]="zero" # adj matrix not connected: 0 (inf: infinity)
config["info"]["set_weight_link_or_dist"]="link" # adj matrix 01 (dist: use weight)
config["info"]["calculate_weight_adj"]=False
# config["info"]["weight_adj_epsilon"]=0.1 # disabled when the above is false

json.dump(config, open(os.path.join(DATA_PATH, DATASET, "config.json"), "w", encoding="utf-8"), ensure_ascii=False)
config

{'geo': {'including_types': ['Point'], 'Point': {}},
 'rel': {'including_types': ['geo'], 'geo': {}},
 'dyna': {'including_types': ['state'],
  'state': {'entity_id': 'geo_id', 'flow': 'num'}},
 'info': {'data_files': 'sz_taxi',
  'geo_file': 'sz_taxi',
  'rel_file': 'sz_taxi',
  'data_col': ['flow'],
  'output_dim': 1,
  'time_intervals': 900,
  'init_weight_inf_or_zero': 'zero',
  'set_weight_link_or_dist': 'link',
  'calculate_weight_adj': False}}